In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re

In [13]:
# now start to change the variable in url

check_in_date = "2024-11-25"
check_out_date = "2024-12-12"
city = "Madrid"
country = "Spain"

adult_num = "1"
romm_num = "1"
children_num = "0"


headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}


url = f"https://www.booking.com/searchresults.en-gb.html?ss={city}%2C+Community+of+{city}%2C+{country}&efdco=1&label=gen173bo-1DCAEoggI46AdICVgDaMgBiAEBmAEJuAEXyAEM2AED6AEB-AEDiAIBmAICqAIDuAKNxYG6BsACAdICJDc4YzI3MmQ4LWM1NzMtNDE4MS04MTk0LTZlMTdkZWNjMWQwZdgCBOACAQ&sid=badcbe9c6b390c4f5668ded61093febe&aid=304142&lang=en-gb&sb=1&src_elem=sb&src=index&dest_id=-390625&dest_type=city&ac_position=0&ac_click_type=b&ac_langcode=en&ac_suggestion_list_length=5&search_selected=true&search_pageview_id=d4b24c866c100605&ac_meta=GhBkNGIyNGM4NjZjMTAwNjA1IAAoATICZW46BE1hZHJAAEoAUAA%3D&checkin={check_in_date}&checkout={check_out_date}&group_adults={adult_num}&no_rooms={romm_num}&group_children={children_num}&order=price"


# request to the website
response = requests.get(url,headers=headers)
html= response.content
soup = BeautifulSoup(html)

# choose container
hotel_container = soup.select("div > div.c624d7469d.a0e60936ad.a3214e5942.b0db0e8ada")

container = []

# filter out ad
for con in hotel_container:
    # Check for ad markers
    if con.find("span", class_="b30f8eb2d6", string="Ad"):
        print("Ad found, skipping...")
        continue
    container.append(con)


container


[<div class="c624d7469d a0e60936ad a3214e5942 b0db0e8ada" style="--bui_stack_spaced_gap--s:2"><div class="c624d7469d f034cf5568 a937b09340 a3214e5942 f02fdbd759" style="--bui_stack_spaced_gap--s:6"><div class="dc5041d860 c72df67c95"><div class="c624d7469d a0e60936ad a3214e5942" style="--bui_stack_spaced_gap--s:1"><div><div class="d6767e681c"><h3 class="aab71f8e4e"><a class="a78ca197d0" data-testid="title-link" href="https://www.booking.com/hotel/es/hostel-thirty-one-31.en-gb.html?label=gen173bo-1DCAEoggI46AdICVgDaMgBiAEBmAEJuAEXyAEM2AED6AEB-AEDiAIBmAICqAIDuAKNxYG6BsACAdICJDc4YzI3MmQ4LWM1NzMtNDE4MS04MTk0LTZlMTdkZWNjMWQwZdgCBOACAQ&amp;sid=badcbe9c6b390c4f5668ded61093febe&amp;aid=304142&amp;ucfs=1&amp;arphpl=1&amp;checkin=2024-11-25&amp;checkout=2024-12-12&amp;dest_id=-390625&amp;dest_type=city&amp;group_adults=1&amp;req_adults=1&amp;no_rooms=1&amp;group_children=0&amp;req_children=0&amp;hpos=1&amp;hapos=1&amp;sr_order=price&amp;srpvid=28e40153255a01e9&amp;srepoch=1732493480&amp;all_sr_bl

In [25]:
# loop container to extract hote name, price, rating, vote

name_selector = "a > div.f6431b446c.a15b38c233"
price_selector = "div > span.f6431b446c.fbfd7c1165.e84eb96b1f"
rating_selector = "div > div.ac4a7896c7"
vote_selector = "div > div.abf093bdfe.f45d8e4c32.d935416c47"
distance_selector = 'span > span[data-testid="distance"]'

name_lst =[]
price_lst = []
rating_lst = []
vote_lst = []
distance_lst = []


# extract hote data
def hotel_extract(item, selector):
    parse = item.select(selector)
    if parse:
        return parse[0].text
    else:
        return np.nan



for con in container:
    #name
    name = hotel_extract(con, name_selector)
    name_lst.append(name)

    #price
    price = hotel_extract(con, price_selector)
    price_lst.append(price)


    #rating
    rating = hotel_extract(con, rating_selector)
    vote = hotel_extract(con, vote_selector)
    if (not "Scored" in rating) and (con.find("span", class_="b30f8eb2d6", string="New to Booking.com")):
        rating = np.nan
        vote = np.nan
    
    rating_lst.append(rating)
    vote_lst.append(vote)
        

    

    #distance
    distance = hotel_extract(con, distance_selector)
    distance_lst.append(distance)

print(name_lst)
print(price_lst)
print(rating_lst)
print(vote_lst)
print(distance_lst)

dic = {}

dic = {"city":city,"country":country,"name":name_lst,"price":price_lst,"rating":rating_lst,"vote":vote_lst,"distance":distance_lst}
df = pd.DataFrame(dic)

# Save the DataFrame to a CSV file
df.to_csv("hotel_data_spain.csv", index=False)
df

['Hostel Thirty One', 'Hostel SOFÍA', 'Hostel sofía', 'Hostel OASIS', 'Hostel Vida Verde', 'Hostel Estrella 20', 'Hostel MYD La Latina', 'Fit Hostel Madrid', 'Arc House Pop Art', 'Motion Chueca Hostel', 'Petit Hostel La Latina', 'Hostel M Torres', 'Bed Madrid Hostel by Bossh! Hotels', 'Hostels Meetingpoint', 'Hostal Siesta & Go Nuevos Ministerios', 'min hostel capsules', 'Madriz Hostel', 'Debod Hostel', 'Madrid Motion Hostels', 'Toc Hostel Madrid', 'The Adventure Hostel Gran Vía', 'Live It Madrid Chamberi', 'Good Vibes Madrid', 'Velvet Hostels Pacifico', 'OXYGEN HOSTEL Capsula']
['SEK\xa03,669', 'SEK\xa03,819', 'SEK\xa03,934', 'SEK\xa03,968', 'SEK\xa04,845', 'SEK\xa04,992', 'SEK\xa05,114', 'SEK\xa05,148', 'SEK\xa05,464', 'SEK\xa05,585', 'SEK\xa05,632', 'SEK\xa05,728', 'SEK\xa06,016', 'SEK\xa06,085', 'SEK\xa06,376', 'SEK\xa06,648', 'SEK\xa06,998', 'SEK\xa07,040', 'SEK\xa07,193', 'SEK\xa07,290', 'SEK\xa07,578', 'SEK\xa09,432', 'SEK\xa09,605', 'SEK\xa010,364', 'SEK\xa010,439']
['Scored 5.

,city,country,name,price,rating,vote,distance
0,Madrid,Spain,Hostel Thirty One,"SEK 3,669",Scored 5.3,"2,188 reviews",3.9 km from centre
1,Madrid,Spain,Hostel SOFÍA,"SEK 3,819",Scored 4.8,601 reviews,6.1 km from centre
2,Madrid,Spain,Hostel sofía,"SEK 3,934",Scored 5.5,"2,504 reviews",6.1 km from centre
3,Madrid,Spain,Hostel OASIS,"SEK 3,968",Scored 6.4,131 reviews,4.3 km from centre
4,Madrid,Spain,Hostel Vida Verde,"SEK 4,845",Scored 5.6,201 reviews,7.7 km from centre
5,Madrid,Spain,Hostel Estrella 20,"SEK 4,992",Scored 7.8,195 reviews,0.7 km from centre
6,Madrid,Spain,Hostel MYD La Latina,"SEK 5,114",Scored 6.2,"2,674 reviews",1.1 km from centre
7,Madrid,Spain,Fit Hostel Madrid,"SEK 5,148",Scored 5.7,202 reviews,1.4 km from centre
8,Madrid,Spain,Arc House Pop Art,"SEK 5,464",Scored 6.3,546 reviews,1.1 km from centre
9,Madrid,Spain,Motion Chueca Hostel,"SEK 5,585",Scored 4.9,"7,066 reviews",0.6 km from centre


In [19]:
name = container[0].select("a > div.f6431b446c.a15b38c233")[0].text
price = container[0].select("div > span.f6431b446c.fbfd7c1165.e84eb96b1f")[0].text
rating = container[0].select("div.a3b8729ab1.d86cee9b25 > div.ac4a7896c7")[0].text
vote = container[0].select("div > div.abf093bdfe.f45d8e4c32.d935416c47")[0].text
distance  = container[0].select('span > span[data-testid="distance"]')[0].text


print(name)
print(price)
print(rating)
print(vote)
print(distance)


name = container[1].select("a > div.f6431b446c.a15b38c233")[0].text
price = container[1].select("div > span.f6431b446c.fbfd7c1165.e84eb96b1f")[0].text
rating = container[1].select("div > div.ac4a7896c7")[0].text
vote = container[1].select("div > div.abf093bdfe.f45d8e4c32.d935416c47")[0].text
distance  = container[1].select('span > span[data-testid="distance"]')[0].text


print(name)
print(price)
print(rating)
print(vote)
print(distance)


name = container[2].select("a > div.f6431b446c.a15b38c233")[0].text
price = container[2].select("div > span.f6431b446c.fbfd7c1165.e84eb96b1f")[0].text
rating = container[2].select("div > div.ac4a7896c7")[0].text
vote = container[2].select("div > div.abf093bdfe.f45d8e4c32.d935416c47")[0].text
distance  = container[2].select('span > span[data-testid="distance"]')[0].text


print(name)
print(price)
print(rating)
print(vote)
print(distance)

Hostel Thirty One
SEK 3,669
Scored 5.3 
2,188 reviews
3.9 km from centre
Hostel SOFÍA
SEK 3,819
Scored 4.8 
601 reviews
6.1 km from centre
Hostel sofía
SEK 3,934
Scored 5.5 
2,504 reviews
6.1 km from centre


In [12]:
df

,city,country,name,price,rating,vote,distance
0,Madrid,Spain,Hostel Acacias,"SEK 3,324",Scored 6.4,"1,398 reviews",1.5 km from centre
1,Madrid,Spain,Hostel Thirty One,"SEK 3,416",Scored 5.3,"2,188 reviews",3.9 km from centre
2,Madrid,Spain,Hostel SOFÍA,"SEK 3,543",Scored 4.8,601 reviews,6.1 km from centre
3,Madrid,Spain,Hostel sofía,"SEK 3,612",Scored 5.5,"2,504 reviews",6.1 km from centre
4,Madrid,Spain,Hostel OASIS,"SEK 3,669",Scored 6.4,131 reviews,4.3 km from centre
5,Madrid,Spain,Hostel Vida Verde,"SEK 4,381",Scored 5.6,201 reviews,7.7 km from centre
6,Madrid,Spain,Hostel Estrella 20,"SEK 4,624",Scored 7.8,195 reviews,0.7 km from centre
7,Madrid,Spain,Fit Hostel Madrid,"SEK 4,637",Scored 5.7,202 reviews,1.4 km from centre
8,Madrid,Spain,Hostel MYD La Latina,"SEK 4,756",Scored 6.2,"2,674 reviews",1.1 km from centre
9,Madrid,Spain,Arc House Pop Art,"SEK 5,015",Scored 6.3,546 reviews,1.1 km from centre
